In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from skimage.io import imread
import matplotlib.pyplot as plt
import pigleg_cv
import shutil

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NOTE! Installing ujson may make loading annotations faster.


2023-12-18 08:30:36,306 - /webapps/piglegsurgery/piglegcv/incision_detection_mmdet.py - DEBUG - mmdet.version=2.25.0
2023-12-18 08:30:36,307 - /webapps/piglegsurgery/piglegcv/incision_detection_mmdet.py - DEBUG - 11.7
2023-12-18 08:30:36,307 - /webapps/piglegsurgery/piglegcv/incision_detection_mmdet.py - DEBUG - GCC 9.4
2023-12-18 08:30:36.309 | DEBUG    | pigleg_cv:<module>:50 - DEVICE=None


1.12.0a0+8a1a93a True


In [2]:
!pwd

/webapps/piglegsurgery/piglegcv


In [3]:
import torch

torch.cuda.is_available()

True

In [4]:
!nvidia-smi

Mon Dec 18 08:30:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:42:00.0 Off |                  N/A |
| 48%   60C    P5    15W / 250W |    266MiB / 11175MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
torch.__version__

'1.12.0a0+8a1a93a'

In [6]:
Path("~/.cache/torch/hub/checkpoints/").expanduser().exists()

True

In [7]:
!ls ~/.cache/torch/hub/checkpoints/

yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth


In [8]:
# pip install mmcv-full

In [6]:
bool("False")


def make_bool_from_string(s: str) -> bool:
    if type(s) == bool:
        return s
    else:
        if s.lower() in ("true", "1"):
            return True
        else:
            return False


out = make_bool_from_string(False)
print(out)
print(type(out))

False
<class 'bool'>


# Run whole video processing

In [9]:
is_microsurgery = False
test_first_seconds = False
n_stitches = 3

patch = "*/Anasto*.mp4"
# patch = "*/2023_10_11_B-Grübbel_Mareike_Fortlaufende_Transkutannaht.mov"
patch = "Einzel.MOV"
# patch = "test_4.mp4"; n_stitches=0
# patch = "micro_01*.mp4"
# patch = "2023_10_11_B_Seltmann_Charlotte_fortlaufende_Transcutannaht.mp4"; n_stitches=5
# patch = "Katharina_Einzelknopfnaht"; n_stitches=3
patch = "Johanna"
n_stitches = 2


img_pths = list(Path("../piglegsurgeryweb/media/upload/").glob(f"*/*{patch}*"))
img_pths[0]

PosixPath('../piglegsurgeryweb/media/upload/20211104-202301_2021-11-03-A-Seifert,Johanna-Einzelknopfnaht am Schweinefuß_4cbf12bdae2fb1ec0e566589276d3d149751773d/2021-11-03-A-SeifertJohanna-Einzelknopfnaht_am_Schweinefuß.mov')

In [10]:
outputdir = "./del_test_video_output/"

# original outputdir
zf = list(Path("../piglegsurgeryweb/media/").glob(f"*/*{patch}*.zip"))[0]
outputdir = (zf.parent / zf.name[:18]).absolute().resolve()
# outputdir = outputdir.absolute().resolve()
outputdir

PosixPath('/webapps/piglegsurgery/piglegsurgeryweb/media/SA_20211104-202301_kjzuoikcdlyw/SA_20211104-202301')

In [11]:
# list(zf.parent.glob("*"))

## Do whole processing together


In [ ]:
dcv = pigleg_cv.DoComputerVision(
    img_pths[0],
    outputdir,
    test_first_seconds=test_first_seconds,
    is_microsurgery=is_microsurgery,
)
dcv.run()

2023-12-18 08:30:51.700 | DEBUG    | pigleg_cv:__init__:79 - device=cuda
2023-12-18 08:30:51.705 | INFO     | pigleg_cv:run:107 - CV processing started on ../piglegsurgeryweb/media/upload/20211104-202301_2021-11-03-A-Seifert,Johanna-Einzelknopfnaht am Schweinefuß_4cbf12bdae2fb1ec0e566589276d3d149751773d/2021-11-03-A-SeifertJohanna-Einzelknopfnaht_am_Schweinefuß.mov, outputdir=/webapps/piglegsurgery/piglegsurgeryweb/media/SA_20211104-202301_kjzuoikcdlyw/SA_20211104-202301
2023-12-18 08:30:51.706 | DEBUG    | pigleg_cv:run_video_processing:242 - Running video processing...
2023-12-18 08:30:51.707 | DEBUG    | pigleg_cv:get_parameters_for_crop_rotate_rescale:339 - device=cuda
2023-12-18 08:30:51.719 | DEBUG    | run_perpendicular:get_frame_to_process:174 - 9429
2023-12-18 08:30:51.754 | DEBUG    | run_perpendicular:get_frame_to_process:185 - Last frame capture error, frame
2023-12-18 08:30:51.770 | DEBUG    | run_perpendicular:get_frame_to_process:185 - Last frame capture error, frame
202

Fusing layers... 
IDetect.fuse
load checkpoint from local path: /webapps/piglegsurgery/piglegcv/resources/incision_detection_models/220326_234659_mmdet.pth


2023-12-18 08:30:55,073 - /webapps/piglegsurgery/piglegcv/incision_detection_mmdet.py - DEBUG - number of detected incisions = 1
2023-12-18 08:30:55.076 | DEBUG    | pigleg_cv:get_parameters_for_crop_rotate_rescale:352 - {'bbox_micro_calibration': [],
 'bbox_scene_area': [4.145229816436768,
                     8.28770923614502,
                     939.9999389648438,
                     619.4143676757812,
                     0.6346746683120728],
 'box': [[809, 210], [842, 267], [903, 238], [870, 179]],
 'incision_bboxes': [[296.5797119140625,
                      387.8492126464844,
                      495.9276123046875,
                      450.55242919921875,
                      0.9324182271957397]],
 'incision_bboxes_old': [[296.0584716796875,
                          387.4178771972656,
                          499.3452453613281,
                          455.0485534667969,
                          0.9962989687919617]],
 'is_detected': True,
 'is_microsurgery': False,
 'p

Fusing layers... 
IDetect.fuse


2023-12-18 08:31:18.185 | DEBUG    | run_qr:get_bboxes:31 - saving single_image_detector result
2023-12-18 08:31:18.268 | DEBUG    | run_qr:get_bboxes:71 - [[     618.72      133.05      693.15      199.83     0.98143]
 [     42.911      200.38      621.41         470    0.058778]
 [     6.0242      12.158      609.01      433.55    0.056236]]
2023-12-18 08:31:18.270 | DEBUG    | run_qr:get_bboxes:82 - micro calibration detected
2023-12-18 08:31:18.271 | DEBUG    | tools:sort_bboxes_and_masks_by_confidence:196 - sorted_indices=array([0, 1]), int64
2023-12-18 08:31:18.429 | DEBUG    | run_qr:bbox_info_extraction_from_frame:125 - qr code text = 'QR scale pigleg'
2023-12-18 08:31:18.431 | DEBUG    | run_qr:bbox_info_extraction_from_frame:211 - {'is_microsurgery': False, 'pix_size_method': 'QR', 'is_detected': True, 'box': [[620, 155], [646, 200], [693, 177], [667, 132]], 'pix_size': 0.0005195190439467303, 'incision_bboxes': [[226.82362365722656, 295.8583068847656, 378.24908447265625, 340.

...

/home/appuser/mmdetection/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
2023-12-18 08:31:20.106 | DEBUG    | tools:save_json:16 - Writing '/webapps/piglegsurgery/piglegsurgeryweb/media/SA_20211104-202301_kjzuoikcdlyw/SA_20211104-202301/stitch_detection_0.json'
2023-12-18 08:31:20.108 | DEBUG    | tools:save_json:25 - old keys: ['stitch_labels', 'stitch_bboxes']
2023-12-18 08:31:20.108 | DEBUG    | tools:save_json:27 - updated keys: ['stitch_labels', 'stitch_bboxes']
2023-12-18 08:31:20,109 - /webapps/piglegsurgery/piglegcv/stitch_detection_mmdet.py - DEBUG - number of detected stitches = 5
2023-12-18 08:31:20,110 - /webapps/piglegsurgery/piglegcv/stitch_detection_mmdet.py - DEBUG - Stitch detection finished, boxes in: /webapps/piglegsurgery/piglegsurgeryweb/media/SA_20211104-202301_kjzuoikcdlyw/SA_20211104

load checkpoint from local path: /webapps/piglegsurgery/piglegcv/resources/tracker_model_bytetrack/epoch.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_backbone_stage1_1_main_conv_bn_running_mean, ema_detector_backbone_stage1

2023-12-18 08:31:26,486 - mmdet - INFO - image shape: height=800, width=1440 in YOLOX.__init__
2023-12-18 08:31:26,511 - /webapps/piglegsurgery/piglegcv/incision_detection_mmdet.py - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth'}
2023-12-18 08:31:26,511 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-12-18 08:31:26,512 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-12-18 08:31:26,875 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, t

load checkpoint from local path: /webapps/piglegsurgery/piglegcv/resources/tracker_model_bytetrack_hands_tools/epoch_2.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_backbone_stage1_1_main_conv_bn_running_mean, ema_detector_b

2023-12-18 08:31:29.831 | DEBUG    | run_tracker_bytetrack:main_tracker_bytetrack:73 - hash_hex='0xbb95362b3c3cd26a53b7dba0eb7b317e55fc489ad4e6b6dd3e94b772d6970c8a86cf37dc0b69d6f4'
2023-12-18 08:31:29.878 | DEBUG    | run_tracker_bytetrack:main_tracker_bytetrack:103 - Tracking on frame 0, 0.00% (0.01s elapsed, 1000.00s remaining)
2023-12-18 08:31:41.382 | DEBUG    | run_tracker_bytetrack:main_tracker_bytetrack:103 - Tracking on frame 50, 0.53% (11.51s elapsed, 2158.36s remaining)
2023-12-18 08:31:52.917 | DEBUG    | run_tracker_bytetrack:main_tracker_bytetrack:103 - Tracking on frame 100, 1.06% (23.05s elapsed, 2148.91s remaining)
2023-12-18 08:32:04.644 | DEBUG    | run_tracker_bytetrack:main_tracker_bytetrack:103 - Tracking on frame 150, 1.59% (34.77s elapsed, 2149.96s remaining)
2023-12-18 08:32:16.266 | DEBUG    | run_tracker_bytetrack:main_tracker_bytetrack:103 - Tracking on frame 200, 2.12% (46.40s elapsed, 2139.79s remaining)
2023-12-18 08:32:27.913 | DEBUG    | run_tracker_byte

In [12]:
# report_data = dcv.make_report()